In [1]:
import pandas as pd
import os
import numpy as np
import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)
# Parameter windowing
input_width = 48
label_width = 48
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)

def import_data(pathload, hari_khusus_saja=False):
    df_awal=pd.read_excel(pathload)
    #DROP KOLOM SUHU
    df_awal.drop(columns=['Keterangan', 'Keterangan.1'], inplace=True)
    df_awal.dropna(subset=['Beban'], inplace=True)
    if hari_khusus_saja==True:
        return df_awal[df_awal['Hari_Khus']!=0].copy()
    else:
        return df_awal.copy()
    
    
def datetime_kolom(data, jumlah_hari, kelipatan_hari):
    df=data.copy()
    df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S", dayfirst=True) #year-month-day(tanggal bukan hari)  H= hours, M=minutes, S= sekon 
    df['Tanggal']=df['Date'].dt.day
    df['Bulan']=df['Date'].dt.month
    df['Tahun']=df['Date'].dt.year
    df['Jam']=df['Date'].dt.hour
    df['Menit']=df['Date'].dt.minute
    
    df['Jam']=df['Jam']+df['Menit']/60 # agar setiap 30 menit menjadi 0.5 jam (contoh jam 13:30 menjadi 13.5)
    df.drop(columns=['Menit'], inplace=True)

    #1 hari = 48 (karena 30 menit), 1 bulan = 30 hari, 1 tahun =12 bulan
    mod_data= df['Beban'].count() % 48
    smpl_dta=df['Date'].count()-df['Beban'].count()+mod_data+(48*jumlah_hari*kelipatan_hari)
    df=df[-smpl_dta:]
    return df.copy()


def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)
def sin_cos_time(data):
    df= data.copy()
    df['Jam_sin'] = np.sin(df['Jam'] * (2 * np.pi / 24))
    df['Jam_cos'] = np.cos(df['Jam'] * (2 * np.pi / 24))
    df['Hari_sin'] = np.sin((df['Hari']-1) * (2 * np.pi / 7))
    df['Hari_cos'] = np.cos((df['Hari']-1) * (2 * np.pi / 7))
    df['Hari_Khus_sin'] = np.sin((df['Hari_Khus']) * (2 * np.pi / 23))
    df['Hari_Khus_cos']= np.cos((df['Hari_Khus']) * (2 * np.pi / 23))
    df['Tanggal_sin'] = np.sin((df['Tanggal']-1) * (2 * np.pi / 30))
    df['Tanggal_cos'] = np.cos((df['Tanggal']-1) * (2 * np.pi / 30))
    df['Bulan_sin'] = np.sin((df['Bulan']-1) * (2 * np.pi / 12))
    df['Bulan_cos'] = np.cos((df['Bulan']-1) * (2 * np.pi / 12))
    return df.copy()
    

d:\KULIAH S2\Komputasi Lanjut\CODE


In [2]:
df_awal                         =   import_data(pathload='cuaca_bebanLR5_yangbaru.xlsx', hari_khusus_saja=False)
df_awal

,Date,JKT_suhu,JKT_hum,JKT_rain,JBR_suhu,JBR_hum,JBR_rain,JTG_suhu,JTG_hum,JTG_rain,JTM_suhu,JTM_hum,JTM_rain,BLI_suhu,BLI_hum,BLI_rain,Hari,Hari_Khus,Beban
0,2022-01-01 00:30:00,25.25,85.5,0.00,21.25,97.5,0.0,23.00,98.0,0.0,24.75,93.0,0.0,25.70,87.0,0.0,6,2,18402.00
1,2022-01-01 01:00:00,25.10,86.0,0.00,21.10,98.0,0.0,22.90,98.0,0.0,24.80,92.0,0.0,25.80,86.0,0.0,6,2,18214.00
2,2022-01-01 01:30:00,25.15,86.0,0.00,21.05,97.0,0.0,23.15,96.5,0.0,24.80,93.0,0.1,25.80,86.5,0.0,6,2,17974.00
3,2022-01-01 02:00:00,25.20,86.0,0.00,21.00,96.0,0.0,23.40,95.0,0.0,24.80,94.0,0.2,25.80,87.0,0.0,6,2,17729.00
4,2022-01-01 02:30:00,25.15,86.0,0.00,20.90,95.5,0.0,23.35,95.5,0.0,24.65,94.5,0.1,25.85,86.5,0.0,6,2,17532.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37079,2024-02-12 12:00:00,30.20,67.0,0.10,26.90,68.0,0.0,29.30,78.0,0.2,31.10,63.0,0.0,29.30,74.0,0.0,1,0,26775.07
37080,2024-02-12 12:30:00,30.60,65.0,0.10,27.50,65.5,0.0,29.45,77.0,0.2,31.60,60.5,0.0,29.50,73.0,0.0,1,0,26803.02
37081,2024-02-12 13:00:00,31.00,63.0,0.10,28.10,63.0,0.0,29.60,76.0,0.2,32.10,58.0,0.0,29.70,72.0,0.0,1,0,27538.69
37082,2024-02-12 13:30:00,31.40,61.0,0.05,28.15,62.5,0.0,29.65,75.5,0.2,32.20,57.0,0.0,29.80,71.5,0.0,1,0,28408.38


In [3]:
df_awal['Hari_Khus'].unique()

array([ 2,  0,  3,  4,  5,  1,  6,  8,  7,  9, 18, 10, 11, 12, 13, 14, 15,
       16, 17, 20, 21, 22, 23], dtype=int64)

In [4]:
df_1=datetime_kolom(data=df_awal, jumlah_hari=365, kelipatan_hari=1)
df_1 =df_1[['Date','Bulan','Tanggal','Hari', 'Jam','Beban', "Hari_Khus"]].copy()
df_1

,Date,Bulan,Tanggal,Hari,Jam,Beban,Hari_Khus
19536,2023-02-12 00:30:00,2,12,7,0.5,20312.00,0
19537,2023-02-12 01:00:00,2,12,7,1.0,20180.00,0
19538,2023-02-12 01:30:00,2,12,7,1.5,20021.00,0
19539,2023-02-12 02:00:00,2,12,7,2.0,19906.00,0
19540,2023-02-12 02:30:00,2,12,7,2.5,19743.00,0
...,...,...,...,...,...,...,...
37079,2024-02-12 12:00:00,2,12,1,12.0,26775.07,0
37080,2024-02-12 12:30:00,2,12,1,12.5,26803.02,0
37081,2024-02-12 13:00:00,2,12,1,13.0,27538.69,0
37082,2024-02-12 13:30:00,2,12,1,13.5,28408.38,0


In [5]:
df_2 = sin_cos_time(data=df_1)
cutoff = pd.to_datetime('2024-01-01 00:00:00')
df_3= df_2[df_2['Date'] <= cutoff].copy()
df_3.drop(columns=['Bulan', 'Tanggal','Hari', 'Jam'], inplace=True)
df_3.set_index('Date', inplace=True)
df_3

,Beban,Hari_Khus,Jam_sin,Jam_cos,Hari_sin,Hari_cos,Hari_Khus_sin,Hari_Khus_cos,Tanggal_sin,Tanggal_cos,Bulan_sin,Bulan_cos
Date,,,,,,,,,,,,
2023-02-12 00:30:00,20312.00,0,0.130526,0.991445,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:00:00,20180.00,0,0.258819,0.965926,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:30:00,20021.00,0,0.382683,0.923880,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:00:00,19906.00,0,0.500000,0.866025,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:30:00,19743.00,0,0.608761,0.793353,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 22:00:00,21241.27,0,-0.500000,0.866025,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 22:30:00,20800.42,0,-0.382683,0.923880,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 23:00:00,20450.02,0,-0.258819,0.965926,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025


In [6]:
df_load=df_3[['Beban']].copy() 
df_4=df_3.copy()
df_4['Beban']=(df_load - df_load.min()) / (df_load.max() - df_load.min())
df_4

,Beban,Hari_Khus,Jam_sin,Jam_cos,Hari_sin,Hari_cos,Hari_Khus_sin,Hari_Khus_cos,Tanggal_sin,Tanggal_cos,Bulan_sin,Bulan_cos
Date,,,,,,,,,,,,
2023-02-12 00:30:00,0.351179,0,0.130526,0.991445,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:00:00,0.343493,0,0.258819,0.965926,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:30:00,0.334235,0,0.382683,0.923880,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:00:00,0.327539,0,0.500000,0.866025,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:30:00,0.318048,0,0.608761,0.793353,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 22:00:00,0.405287,0,-0.500000,0.866025,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 22:30:00,0.379618,0,-0.382683,0.923880,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 23:00:00,0.359216,0,-0.258819,0.965926,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025


In [7]:
x,y=make_windows(df_4.drop(columns=["Beban"]).to_numpy(), df_4['Beban'].to_numpy(),total_window_size, input_slice, labels_slice)
x.shape, y.shape

((15409, 48, 11), (15409, 48))

In [9]:
import tensorflow as tf

input_width=x.shape[1]
num_features=x.shape[2]
num_output = 1
out_steps=  OUT_STEPS
model_td = tf.keras.models.Sequential()

model_td.add(tf.keras.layers.LSTM(units=64, input_shape=(input_width, num_features), return_sequences=True))
model_td.add(tf.keras.layers.LSTM(units=64, return_sequences=False))
model_td.add(tf.keras.layers.Dense(out_steps*num_output,   kernel_initializer=tf.initializers.zeros()))
model_td.add(tf.keras.layers.Reshape([out_steps, num_output]))
# Compile the model using an optimizer and a loss function appropriate for regression (MSE)
model_td.compile(optimizer='adam', loss='mse')

# Display model architecture summary
model_td.summary()

# Train the model
# X_train: shape (15409, 48, 11)
# Y_train: shape (15409, 48)
history = model_td.fit(
    x,        # Your input training data
    y,        # Your target training data
    batch_size=48,  # Number of samples per gradient update
    epochs=100,      # Number of training epochs
    validation_split=0.2,  # Reserve 20% of the data for validation
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],
    shuffle=False
)
model_td.save('model_LSTM.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 48, 64)            19456     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 48)                3120      
                                                                 
 reshape (Reshape)           (None, 48, 1)             0         
                                                                 
Total params: 55,600
Trainable params: 55,600
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
257/257 [==============================] - 14s 11ms/step - loss: 0.0381 - val_loss: 0.0184
Epoch 2/100
257/257 [==============================] - 2s 10ms/step - loss: 0.015

In [10]:
# import tensorflow as tf

# OUT_STEPS = 48  # Number of time steps you want to predict

# model_td = tf.keras.Sequential([
#     # Extract the last time step: shape becomes [batch, 1, 11]
#     tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
#     # Dense layer outputs OUT_STEPS predictions
#     tf.keras.layers.Dense(OUT_STEPS, kernel_initializer=tf.initializers.zeros()),
#     # Reshape from [batch, 1, OUT_STEPS] to [batch, OUT_STEPS]
#     tf.keras.layers.Reshape([OUT_STEPS])
# ])

# # Explicitly build the model to avoid the error
# model_td.build((None, 48, 11))
# model_td.summary()

# model_td.compile(optimizer='adam', loss='mse')

# # Assuming x and y are your training data arrays with shapes (15409, 48, 11) and (15409, 48) respectively
# history = model_td.fit(
#     x,                # input training data
#     y,                # target training data
#     batch_size=48,    # samples per batch
#     epochs=20,        # number of epochs
#     validation_split=0.2,  # use the last 20% of data for validation (ensure shuffle=False for time series)
#     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],
#     shuffle=False     # do not shuffle to preserve time order
# )


In [11]:
x[-3081:].shape

(3081, 48, 11)

In [12]:
x[slice(None,None,label_width), :, :].shape

(322, 48, 11)

In [13]:
Beban_min = df_load.min().to_numpy()
Beban_max = df_load.max().T.to_numpy()
prediksi= model_td.predict(x[slice(None,None,label_width), :, :])
print(prediksi)

11/11 [==============================] - 0s 3ms/step
[[[0.54153204]
  [0.52132267]
  [0.5026437 ]
  ...
  [0.71905726]
  [0.6891944 ]
  [0.66153425]]

 [[0.58325887]
  [0.56354856]
  [0.54734397]
  ...
  [0.67915493]
  [0.6500651 ]
  [0.62328327]]

 [[0.5689396 ]
  [0.55069125]
  [0.53589404]
  ...
  [0.6327134 ]
  [0.6064852 ]
  [0.58221066]]

 ...

 [[0.5199931 ]
  [0.5036448 ]
  [0.48835745]
  ...
  [0.551254  ]
  [0.5268633 ]
  [0.50646746]]

 [[0.50733364]
  [0.49100408]
  [0.4746487 ]
  ...
  [0.5054492 ]
  [0.48318213]
  [0.46398318]]

 [[0.46970332]
  [0.45512143]
  [0.4392381 ]
  ...
  [0.4780053 ]
  [0.45852822]
  [0.44241032]]]


In [14]:
prediksi_inv= prediksi.reshape(-1,)*(Beban_max - Beban_min) + Beban_min
print(prediksi_inv.shape)

(15456,)


In [32]:
y_inv= y[slice(None,None,label_width)].reshape(-1)*(Beban_max - Beban_min) + Beban_min

In [36]:
dict_data={'prediksi':prediksi_inv, 'actual':y_inv}
pd.DataFrame(dict_data)

,prediksi,actual
0,23581.191616,19244.00
1,23234.108383,19141.00
2,22913.308958,18943.00
3,22623.413174,18862.00
4,22377.406824,18639.00
...,...,...
15451,23359.036317,21241.27
15452,22893.941079,20800.42
15453,22490.159713,20450.02
15454,22155.653317,20175.87


In [37]:
dict_data={'prediksi':prediksi_inv, 'actual':df_3['Beban'].to_numpy()[48:]}
pd.DataFrame(dict_data)

,prediksi,actual
0,23581.191616,19244.00
1,23234.108383,19141.00
2,22913.308958,18943.00
3,22623.413174,18862.00
4,22377.406824,18639.00
...,...,...
15451,23359.036317,21241.27
15452,22893.941079,20800.42
15453,22490.159713,20450.02
15454,22155.653317,20175.87


In [16]:
y[-1:]*(Beban_max - Beban_min) + Beban_min

array([[21473.45, 21138.82, 20921.34, 20705.89, 20574.73, 20392.16,
        20407.34, 20554.4 , 20859.61, 21096.81, 20846.87, 20322.48,
        19949.12, 19759.25, 19845.38, 20142.65, 20413.43, 20714.54,
        21093.  , 21343.11, 21599.32, 21594.56, 21691.83, 21684.06,
        21779.6 , 21828.59, 21945.7 , 21856.56, 21842.73, 21781.67,
        21849.13, 21841.02, 21999.71, 22153.32, 22594.13, 23537.32,
        24036.42, 23611.48, 23385.53, 22891.6 , 22477.12, 22137.57,
        21697.63, 21241.27, 20800.42, 20450.02, 20175.87, 19802.04]])